In [36]:
import os
import sys

# Add src directory to Python path
src_path = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.insert(0, src_path)

print("Added to path:", src_path)



Added to path: /Users/sukainaalkhalidy/Desktop


In [37]:
import os
import sys
from dotenv import load_dotenv
load_dotenv() 

# Set the target folder name you want to reach
target_folder = "NCEAS_Unsupervised_NLP"

# Get the current working directory
current_dir = os.getcwd()

# Loop to move up the directory tree until we reach the target folder
while os.path.basename(current_dir) != target_folder:
    parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
    if parent_dir == current_dir:
        # If we reach the root directory and haven't found the target, exit
        raise FileNotFoundError(f"{target_folder} not found in the directory tree.")
    current_dir = parent_dir

# Change the working directory to the folder where "phate-for-text" is found
os.chdir(current_dir)

# Add the "phate-for-text" directory to sys.path
sys.path.insert(0, current_dir)
sys.path.insert(0, os.path.join(current_dir, "src"))

In [38]:
# Add the repo root
sys.path.insert(0, current_dir)

# Add src so custom_packages can be found
sys.path.insert(0, os.path.join(current_dir, "src"))



In [43]:
import os
print(os.getcwd())


/Users/sukainaalkhalidy/Desktop/CMSE 495 capstone/NCEAS_Unsupervised_NLP


In [44]:
import pandas as pd
df = pd.read_csv("src/data/arxiv/data/arxiv/arxiv_30k_clean.csv")

df.head()


,text,label
0,Semantic Agreement Enables Efficient Open-Ende...,cs.CL
1,Scheduling in Grid Computing Environment Sch...,cs.DC
2,Taking off the Rose-Tinted Glasses: A Critical...,cs.LG
3,Traffic Performance Score for Measuring the Im...,physics.soc-ph
4,SueNes: A Weakly Supervised Approach to Evalua...,cs.CL


In [45]:
df.head(1)

,text,label
0,Semantic Agreement Enables Efficient Open-Ende...,cs.CL


In [47]:


df_new = pd.DataFrame()

df_new["topic"] = df["text"]

# Extract primary category
df_new["category 1"] = df["label"]

# Extract top-level category (cs, physics, etc.)
df_new["category 0"] = df["label"].apply(lambda x: x.split(".")[0])



In [48]:
df_new = pd.DataFrame(new)

In [49]:
df_new

""


In [ ]:
NCEAS_Unsupervised_NLP

In [52]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

def get_embeddings(texts, model_name="Qwen/Qwen3-Embedding-0.6B"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    model = SentenceTransformer(model_name, device=device)

    embeddings = model.encode(
        texts,
        batch_size=32,        # Increase if strong GPU
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True
    )

    return embeddings


In [54]:
df_new = pd.DataFrame()

df_new["topic"] = df["text"]
df_new["category 1"] = df["label"]
df_new["category 0"] = df["label"].apply(lambda x: x.split(".")[0])

df_new.head()


,topic,category 1,category 0
0,Semantic Agreement Enables Efficient Open-Ende...,cs.CL,cs
1,Scheduling in Grid Computing Environment Sch...,cs.DC,cs
2,Taking off the Rose-Tinted Glasses: A Critical...,cs.LG,cs
3,Traffic Performance Score for Measuring the Im...,physics.soc-ph,physics
4,SueNes: A Weakly Supervised Approach to Evalua...,cs.CL,cs


In [61]:
embedding_model = "Qwen3-Embedding-0.6B"

os.makedirs(f"{embedding_model}_results", exist_ok=True)
os.makedirs("qwen_embeddings", exist_ok=True)

embedding_list = get_embeddings(df_new["topic"].tolist())

np.save("qwen_embeddings/WOB_qwen_embed.npy", embedding_list)


Using device: cpu


Loading weights:   0%|          | 0/310 [00:00<?, ?it/s]

Batches:   0%|          | 0/938 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
# sample_size = int(0.20 * len(topic_data))
# topic_data_sample = topic_data.iloc[:sample_size].reset_index(drop=True)
# data_sample = data[:sample_size]

In [12]:
topic_dict = {}
for col in topic_data.columns:
    if re.match(r'^category \d+$', col): 
        unique_count = len(topic_data[col].unique())
        topic_dict[unique_count] = np.array(topic_data[col])

In [17]:
reducer_model = phate.PHATE(n_jobs=-2,random_state=42, n_components=300,decay=20,t="auto",n_pca=None)
embed_phate = reducer_model.fit_transform(data)


Calculating PHATE...
  Running PHATE on 46985 observations and 3072 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 687.58 seconds.
    Calculating affinities...
    Calculated affinities in 567.51 seconds.
  Calculated graph and diffusion operator in 1255.41 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 7.65 seconds.
    Calculating KMeans...
    Calculated KMeans in 2.90 seconds.
  Calculated landmark operator in 11.81 seconds.
  Calculating optimal t...
    Automatically selected t = 31
  Calculated optimal t in 0.72 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.57 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 149.19 seconds.
Calculated PHATE in 1417.88 seconds.


In [ ]:
dataset_name = "arxiv"
embedding_model = "Qwen3-Embedding-0.6B"


In [31]:

np.save(
    f"{embedding_model}_reduced_embeddings/PHATE_{dataset_name}_embed.npy",
    np.array(embed_phate)
)


In [13]:
depth= 2
cluster_levels=[]
for i in reversed(range(0, depth)):
    cluster_levels.append(len(topic_data[f'category {i}'].unique()))

In [20]:
include_pca =True
include_umap=True

# Load your embeddings
embeddings = np.array(data)
embedding_methods = {}
# PCA to 2D

embedding_methods["PHATE"]  =embed_phate


In [15]:
np.save(
    f"{embedding_model}_reduced_embeddings/PCA_{dataset_name}_embed.npy",
    embedding_methods["PCA"]
)

np.save(
    f"{embedding_model}_reduced_embeddings/UMAP_{dataset_name}_embed.npy",
    embedding_methods["UMAP"]
)

np.save(
    f"{embedding_model}_reduced_embeddings/tSNE_{dataset_name}_embed.npy",
    embedding_methods["tSNE"]
)


In [18]:
pca_path = os.path.join(
    reduced_folder,
    f"PCA_{dataset_name}_embeddings.npy"
)

umap_path = os.path.join(
    reduced_folder,
    f"UMAP_{dataset_name}_embeddings.npy"
)

# LOAD REDUCED EMBEDDINGS
embedding_methods = {}

if os.path.exists(pca_path):
    embedding_methods["PCA"] = np.load(pca_path)
    print("Loaded PCA embeddings.")
else:
    print(f"PCA file not found at: {pca_path}")

if os.path.exists(umap_path):
    embedding_methods["UMAP"] = np.load(umap_path)
    print("Loaded UMAP embeddings.")
else:
    print(f"UMAP file not found at: {umap_path}")

print("Available embedding methods:", list(embedding_methods.keys()))


In [21]:
from collections import defaultdict
import numpy as np

scores_all = defaultdict(lambda: defaultdict(list))

for embed_name, embed_data in embedding_methods.items():
    for cluster_method in ["Agglomerative", "HDBSCAN", "DC"]:
        for level in cluster_levels:
            
            # -----------------------
            # Clustering
            # -----------------------
            
            if cluster_method == "Agglomerative":
                model = AgglomerativeClustering(n_clusters=level)
                model.fit(embed_data)
                labels = model.labels_
                
            elif cluster_method == "HDBSCAN":
                model = hdbscan.HDBSCAN(min_cluster_size=level)
                model.fit(embed_data)
                labels = model.labels_
                
                # Handle noise (-1)
                labels[labels == -1] = labels.max() + 1
                
            elif cluster_method == "DC":
                model = dc(
                    min_clusters=level,
                    max_iterations=5000,
                    k=10,
                    alpha=4,
                    t=3
                )
                model.fit(embed_data)
                labels = model.labels_
                
            # Match hierarchy level
            
            available_levels = sorted(topic_dict.keys())
            closest_level = min(
                available_levels,
                key=lambda k: abs(k - level)
            )

            topic_series = topic_dict[closest_level]
            valid_idx = ~pd.isna(topic_series)

            target_lst = topic_series[valid_idx]
            label_lst = labels[valid_idx]
            
            # Metrics

            try:
                fm_score = FowlkesMallows.Bk(
                    {level: target_lst},
                    {level: label_lst}
                )[level]['FM']
            except:
                fm_score = np.nan

            scores_all[(embed_name, cluster_method)]["FM"].append(fm_score)
            scores_all[(embed_name, cluster_method)]["Rand"].append(
                rand_score(target_lst, label_lst)
            )
            scores_all[(embed_name, cluster_method)]["ARI"].append(
                adjusted_rand_score(target_lst, label_lst)
            )


In [22]:
rows = []

for (embed_name, cluster_method), score_dict in scores_all.items():
    n_levels = len(score_dict["FM"])  # assumes all metrics have same length
    
    for i in range(n_levels):
        rows.append({
            "reduction_method": embed_name,
            "cluster_method": cluster_method,
            "level": cluster_levels[i],
            "FM": score_dict["FM"][i],
            "Rand": score_dict["Rand"][i],
            "ARI": score_dict["ARI"][i],
            "Params": "{'k':10,'alpha':4,'t':3}"
        })

# Create DataFrame
scores_df = pd.DataFrame(rows)

# Sort for readability
scores_df = scores_df.sort_values(
    by=["reduction_method", "cluster_method", "level"]
).reset_index(drop=True)

# Use dataset_name instead of hardcoded WOS
output_path = f"{embedding_model}_results/other_{dataset_name}_results.csv"

write_header = not os.path.exists(output_path)

scores_df.to_csv(
    output_path,
    mode='a',
    index=False,
    header=write_header
)


In [ ]:
import json
with open("combo_color_map.json", 'r') as file:
        combo_color_map = json.load(file)

In [ ]:
import matplotlib.pyplot as plt

metrics = ['FM', 'Rand', 'ARI']

for metric in metrics:
    plt.figure(figsize=(10, 6))

    for (embed_name, method), metric_scores in scores_all.items():

        display_method = "Diffusion Condensation" if method == "DC" else method
        combo_key = f"{embed_name}_{method}"

        plt.plot(
            cluster_levels,
            metric_scores[metric],
            marker='o',
            label=f"{embed_name} {display_method}",
            color=combo_color_map.get(combo_key, 'black')
        )

    plt.title(f"{metric} Score Across Cluster Levels")
    plt.xlabel("Cluster Level")
    plt.ylabel(metric)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
